## Deploy your pre-trained keras model to AWS
adapted from https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

## 0. Upload your model to SageMaker
This is a manual step.


## 1. input the model names
complete either:
- MODEL_LOCATION.  or
- both of JSON and WEIGHTS_LOCATION

In [2]:
import numpy as np

# if your model is saved as only a .h5 file
MODEL_LOCATION ='model927.h5'

# or if your model is saved as 2 files: model as a .json file, and weights as a .h5 file
JSON_LOCATION = ''
WEIGHTS_LOCATION = ''

## 2. Load Your Model
Simply run the cell below; the model will be loaded based on how you defined the above

In [5]:
def top_3_accuracy(y_true, y_pred): #specific for the doodle model as custom object
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from keras.models import load_model
    model = load_model(MODEL_LOCATION, custom_objects={"top_3_accuracy": top_3_accuracy}) #load the model
    print("loaded model from MODEL_LOCATION")
    
elif JSON_LOCATION!='': # you have your model saved as a JSON file AND weights
#adapted from https://machinelearningmastery.com/save-load-keras-deep-learning-models/
    from keras.models import model_from_json
    json_file = open(JSON_LOCATION, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(WEIGHTS_LOCATION)
    print("loaded model from JSON_LOCATION and WEIGHTS_LOCATION")

loaded model from MODEL_LOCATION


## 3. Convert the Keras Model to the format AWS wants
- Converts to a Protobuff file
- Saves it in a certain aws file structure
- Tarballs this file and zips it

In [9]:
def convert_h5_to_aws(loaded_model):
    """
    Given a pre-trained keras model, this function converts it to a TF SavedModel format
    and saves it in the file structure which AWS expects.
    """
    import tensorflow as tf
    
    # This is the file structure which AWS expects. Cannot be changed.
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    # Save the entire model as a SavedModel.
    loaded_model.save(export_dir, save_format='tf')
    # Create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)

convert_h5_to_aws(model)


INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


## 4. Move the tarball (tar.gz) to S3

In [10]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

This is the name of the bucket which SageMaker made in S3

In [11]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-eu-west-1-800245023913'

## 5. Create a SageMaker Model
First, create an empty train.py file (TensorFlowModel expects this at its 'entry point', but can be empty)

In [12]:
!touch train.py #create an empty python file

In [16]:
import boto3, re
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '2.11',
                                  entry_point = 'train.py')

## 6a) Host the SageMaker model and
## 6b) Create an Endpoint to access the model 

Deploy the model. This can take ~10 minutes

Ignore the message `update_endpoint is a no-op in sagemaker>=2`

In [17]:
# Deploy a SageMaker to an endpoint
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t3.medium')

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----!

In [15]:
# What is our endpoint called?
#endpoint = predictor.endpoint
#endpoint

## Success! You have deployed a keras model into AWS
# ---------------------
### 7. Confirm its working correctly by making a prediction
Now, we want to use our endpoint/model. Create a predictor which uses the endpoint

This step depends on what inputs your model is expecting.

Before deploying to aws, I got the predictions of my model - __locally__ - so that I could compare the local vs aws results (they should be the same). 


In [13]:
# Create a predictor which uses this new endpoint
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

endpoint = 'sagemaker-tensorflow-serving-2023-05-28-11-35-10-856' #get endpoint name from SageMaker > endpoints

predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint, sagemaker_session)
# .predict send the data to our endpoint
data = np.asarray([[5. , 3.5, 1.3, 0.3]]) #<-- update this to have inputs for your model
predictor.predict(data)

{'predictions': [[0.999309, 0.000690674, 1.57288e-16]]}

## Cleanup!

else you will incur extra charges

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

- Stop Notebook
- delete endpoints
- delete models
- delete S3 bucket
- delete cloudwatch groups